In [ ]:
#https://github.com/ZFTurbo/volumentations
import glob
import os
import skimage
import numpy as np
import matplotlib.pyplot as plt
import gc
import pandas as pd
#import albumentations as A
import random
from scipy.ndimage import zoom
from skimage.exposure import match_histograms
from skimage import filters, io, img_as_ubyte, img_as_float32
from skimage import data
from skimage import exposure
import h5py
from skimage.filters import threshold_otsu
from skimage.measure import label
from skimage.morphology import remove_small_objects
from scipy import ndimage as ndi
from vedo import *
import pyvista as pv
from skimage.morphology import skeletonize
from sklearn.metrics import precision_score, recall_score, f1_score

#results from rootine
#rootlength[pix]
#123723.442
#186607.196

In [ ]:
import os
#os.chdir('/home/pythia/Chiron/rhar4542/MAXIM/') #in theory we only need to change these two paths and just copy over the data 
base_path = '/home/pythia/Chiron/rhar4542/MAXIM/'

###  Note:  I split the images down the middle (a and b) to run the 3D Unet (memory issues) and then stitched them back together 

In [ ]:
from pathlib import Path
from os import listdir
from os.path import isfile, join
import pandas as pd
rootdir = Path('/home/pythia/Chiron/rhar4542/MAXIME/split_tiffs')
file_list = [f for f in listdir(rootdir) if isfile(join(rootdir, f))]
file_list=pd.DataFrame(data=file_list)
file_list.columns = ['Data_Path']
file_list = file_list.map(str)
file_list['TEXT_1'] = file_list['Data_Path'].str.split('_grayscale').str[0]
file_list

In [ ]:
UNIQUE_ID =np.unique(file_list['TEXT_1'].astype(str))
len(UNIQUE_ID)

In [ ]:
UNIQUE_ID[3]

In [ ]:
#UNIQUE_ID=UNIQUE_ID[UNIQUE_ID == "SPP_P21_SCE2_C13_t21"]
#UNIQUE_ID

In [ ]:
OUT='/home/pythia/Chiron/rhar4542/MAXIME/PYTHON_OUT/bottom_layer/'
MASK_DIR='/home/pythia/Chiron/rhar4542/MAXIME/mask_tiff/'

###  Note:  [270:914] relates the area where Maxime exctracted washed and measured the roots 

| ImageResolution | 0.0045    |  z-slices of cut-out volumes  |                               |                               |                               | WinRhizo   (Root Washing) |           | X-ray   CT scanning |           |                 |           |
|-----------------|-----------|:-----------------------------:|:-----------------------------:|:-----------------------------:|:-----------------------------:|:-------------------------:|:---------:|:-------------------:|:---------:|:---------------:|:---------:|
|     Columns     | Soil type | lower_slice      layer_bottom | upper_slice      layer_bottom | lower_slice      layer_bottom | upper_slice      layer_bottom |      RootLength [cm]      |           |  RootLength  [vox]  |           | RootLength [cm] |           |
|                 |           |                               |                               |                               |                               |         layer_bot         | layer_top |      layer_bot      | layer_top |    layer_bot    | layer_top |
|       C13       |   Loam    |              270              |              914              |              1271             |              1915             |            1211           |    1723   |        207728       |   201183  |       935       |    905    |
|       C30       |   Loam    |              270              |              914              |              1271             |              1915             |            415            |    524    |        75789        |   78253   |       341       |    352    |
|       C13       |   Sand    |              270              |              914              |              1271             |              1915             |            433            |    952    |        114964       |   160771  |       517       |    723    |
|       C27       |   Sand    |              270              |              914              |              1271             |              1915             |             66            |    367    |        21448        |   78806   |        97       |    355    |

In [ ]:
#top section 
for i in UNIQUE_ID:
    print (i)
    unet_im_a=skimage.io.imread(str(rootdir)+"/"+i+"_grayscale.tif_imga_predictions.h5.tif")
    unet_im_b=skimage.io.imread(str(rootdir)+"/"+i+"_grayscale.tif_imgb_predictions.h5.tif")
    unet_stacked  =np.concatenate((unet_im_a, unet_im_b), axis=0)
    unet_stacked= unet_stacked[270:914]
    skimage.io.imsave(OUT+i+"_8bit_STACKED.tif", unet_stacked, check_contrast=False)
    thresh = threshold_otsu(unet_stacked[200:400]) -40
    #thresh = threshold_otsu(unet_stacked[500:800]) +60 # SPP_P21_SCE2_C27 ?? this one not so good
    print (thresh);
    unet_stacked = (unet_stacked > thresh).astype('ubyte')
    del unet_im_a
    del unet_im_b
    gc.collect()
    
    # unet_stacked_1 =unet_stacked[0:350]
    # unet_stacked_2 =unet_stacked[350:700]
    
    # unet_stacked_1=label(unet_stacked_1)
    # min_size_coef=4000
    # print("min_size_coef   "+str(min_size_coef))

    # unet_stacked_2=label(unet_stacked_2)
    # min_size_coef=4000
    # print("min_size_coef   "+str(min_size_coef))

    # unet_stacked  =np.concatenate((unet_stacked_1, unet_stacked_2), axis=0) 
    
    # unet_stacked = remove_small_objects(unet_stacked, min_size=min_size_coef, connectivity=2)
    # del unet_stacked_1
    # del unet_stacked_2
    # gc.collect
    
    skimage.io.imsave(OUT+i+"_binary_STACKED.tif", unet_stacked, check_contrast=False)

    #lazy
    if i == "SPP_P21_SCE1_C13_t21":
        gt_mask=skimage.io.imread(MASK_DIR+"SPP_P21_SCE1_C13_t21_root_system.tif")
    elif i == "SPP_P21_SCE1_C30_t21": 
        gt_mask=skimage.io.imread(MASK_DIR+"SPP_P21_SCE1_C30_t21_root_system.tif")
    elif i == "SPP_P21_SCE2_C13_t21": 
        gt_mask=skimage.io.imread(MASK_DIR+"SPP_P21_SCE2_C13_t21_root_system.tif")
    elif i == "SPP_P21_SCE2_C27": 
        gt_mask=skimage.io.imread(MASK_DIR+"SPP_P21_SCE2_C27_t21_root_system.tif")

    gt_mask= gt_mask[270:914]
    fig, ax = plt.subplots(1, 2, figsize=[10, 4]);
    ax[0].imshow(gt_mask[500]);
    plt.title ("rootine");
    
    ax[1].imshow(unet_stacked[500]);
    plt.title ("unet");

    unet_root_VOL = Volume(unet_stacked, spacing=(1,1,1))
    unet_root_ISO = unet_root_VOL.isosurface(1)
    unet_root_ISO.write(OUT+i+"_UNET.stl")
    unet_root_STL=pv.read(OUT+i+"_UNET.stl")

    rootine_root_VOL = Volume(gt_mask, spacing=(1,1,1))
    rootine_root_ISO = rootine_root_VOL.isosurface(1)
    rootine_root_ISO.write(OUT+i+"rootine.stl")
    rootine_root_STL=pv.read(OUT+i+"rootine.stl")

    pv.set_plot_theme("document");
    p = pv.Plotter();
    #p.add_title(i, font_size=18, color=None, font=None, shadow=False)
    p.add_mesh(rootine_root_STL, color="grey", opacity=1).rotate_y(90);
    #p.add_mesh(unet_root_STL, color="grey", opacity=0.6);
    p.window_size = 300, 300;
    p.show(jupyter_backend='static');
    p.screenshot(OUT+i+"rootine_3D_RENDER.png") ;

    pv.set_plot_theme("document");
    p = pv.Plotter();
    #p.add_title(i, font_size=18, color=None, font=None, shadow=False)
    p.add_mesh(unet_root_STL, color="red", opacity=1).rotate_y(90);
    p.window_size = 300, 300;
    p.show(jupyter_backend='static');
    p.screenshot(OUT+i+"unet_3D_RENDER.png") ;
    
    
    pv.set_plot_theme("document");
    p = pv.Plotter();
    #p.add_title(i, font_size=18, color=None, font=None, shadow=False)
    p.add_mesh(rootine_root_STL, color="grey", opacity=1).rotate_y(90);
    #p.add_mesh(CELL_STL, color="green", opacity=0.5);
    p.add_mesh(unet_root_STL, color="red", opacity=0.6).rotate_y(90);
    p.window_size = 300, 300;
    p.show(jupyter_backend='static');
    p.screenshot(OUT+i+"both_3D_RENDER.png") ;



 
    ##Lower area
    #skeltonize rootine
    rootine_skeleton = skeletonize(gt_mask)
    data_rootine_skeleton = np.array(rootine_skeleton)
    data_rootine_skeleton=data_rootine_skeleton.flatten()
    data_rootine_skeleton=data_rootine_skeleton[data_rootine_skeleton != 0]

    #skeltonize unet
    unet_binary_pot_skeleton = skeletonize(unet_stacked)
    data_unet = np.array(unet_binary_pot_skeleton)
    data_unet=data_unet.flatten()
    data_unet=data_unet[data_unet != 0]

    #How far off?
    print ("python results from rootine v2 mask= "+ str(int(len(data_rootine_skeleton))))
    print ("python results from 3Dunet= "+ str(int(len(data_unet))))
    print ("the 3D unet is "+ str(((int(len(data_unet)) - int(len(data_rootine_skeleton)))/int(len(data_rootine_skeleton))) *100) + "% off")
    # del rootine_skeleton
    # del unet_binary_pot_skeleton

    data_rootine_skeleton=pd.DataFrame(data_rootine_skeleton)
    data_rootine_skeleton.to_csv(OUT+i+"_"+"layer_bot_data_rootine_skeleton.csv") 

    
    data_unet=pd.DataFrame(data_unet)
    data_unet.to_csv(OUT+i+"_"+"layer_bot_data_unet_skeleton.csv") 


    
    gc.collect()

    del gt_mask
    del unet_stacked
    del unet_root_VOL 
    del unet_root_ISO 
    del unet_root_STL
    del rootine_root_VOL 
    del rootine_root_ISO 
    del rootine_root_STL
    gc.collect()
    
        
    
        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [ ]:
# #memory crashes so just testing F1 on sub volume
# unet_binary_pot=unet_stacked[300:400]
# rootine=gt_mask[300:400]
# flat_predictions = unet_binary_pot.flatten()
# flat_ground_truths = rootine.flatten()
# f1 = f1_score(flat_ground_truths, flat_predictions, average='binary')

# precision = precision_score(flat_ground_truths, flat_predictions, average='binary')  # or 'micro', 'weighted', etc.
# recall = recall_score(flat_ground_truths, flat_predictions, average='binary') 

# qc_metrics = {'QC_test': ['precision', 'recall', 'f1'],
#         'QC_result': [precision, recall, f1]}
# qc_metrics = pd.DataFrame(qc_metrics)